# 4 register business logic to fiware on minikube

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/minikube/env

## register `cmd-proxy` as a subscriber of gamepad  device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${GAMEPAD_ID}.*",
      "type": "${GAMEPAD_TYPE}"
    }],
    "condition": {
      "attrs": ["button"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cmd-proxy:8888/gamepad/"
    },
    "attrs": ["button"]
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5b4f0bce4d2c226e03a34f7d
fiware-correlator: 0ecc000a-8a6f-11e8-8b4a-ae8577baef60
date: Wed, 18 Jul 2018 09:43:42 GMT
x-envoy-upstream-service-time: 3
server: envoy
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5b557aaa4520a8be4c383feb",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "gamepad.*",
          "type": "gamepad"
        }
      ],
      "condition": {
        "attrs": []
      }
    },
    "notification": {
      "timesSent": 2,
      "lastNotification": "2018-07-23T06:52:32.00Z",
      "attrs": [
        "button"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus:5050/notify"
      },
      "lastSuccess": "2018-07-23T06:52:32.00Z"
    }
  },
  {
    "id": "5b557e456beb6f7908e15a2c",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "gamepad.*",
          "type": "gamepad"
        }
      ],
      "condition": {
        "attrs": [
          "button"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2018-07-23T07:05:41.00Z",
      "attrs": [
        "button"
      ],
      "attrsFormat": "normalized",
      "http": {
        "url": "http://cmd-proxy:8888/gamepad/"
      }
    }
  }
]
```

## confirm the sent command to robot when pressed the button of gamepad

### prepare a command to emulate gamepad action

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+0900')
echo "mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${GAMEPAD_TYPE}/${GAMEPAD_ID}/attrs -m \"${d}|button|triangle\""

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### emulate to press a gamepad button

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate gamepad action`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.

example)
```
Client mosqsub|39601-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/gamepad/gamepad/attrs', ... (51 bytes))
2018-07-18T18:52:23.1531907543+0900|button|triangle
Client mosqsub|39601-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/robot/turtlebot3/cmd', ... (24 bytes))
turtlebot3@move|triangle
```

**`turtlebot3@move|triangle` was sent from cmd-proxy.**

### confirm gamepad entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${GAMEPAD_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${GAMEPAD_ID}/ | jq .

example)
```json
{
  "id": "gamepad",
  "type": "gamepad",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T18:52:23.1531907543+0900",
    "metadata": {}
  },
  "button": {
    "type": "string",
    "value": "triangle",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T18:52:23.1531907543+0900"
      }
    }
  }
}
```

### confirm robot entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${ROBOT_ID}/ | jq .

`move_status` is `PENDING`

example)
```json
{
  "id": "turtlebot3",
  "type": "robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T09:53:47.00Z",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "executed square command",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T09:15:26.616Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T09:53:47.817Z"
      }
    }
  },
  "theta": {
    "type": "float32",
    "value": "0.4",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "x": {
    "type": "float32",
    "value": "0.1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.2",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "z": {
    "type": "float32",
    "value": "0.3",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### prepare a command to emulate receiving command result

In [ ]:
echo "mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${ROBOT_TYPE}/${ROBOT_ID}/cmdexe -m \"${ROBOT_ID}@move|executed triangle command\""

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### send message to robot cmdexe topic

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate receiving command result`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.
example)
```
Client mosqsub|39703-Nobuyukin received PUBLISH (d0, q0, r0, m0, '/robot/turtlebot3/cmdexe', ... (41 bytes))
turtlebot3@move|executed triangle command
```

### confirm robot entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${ROBOT_ID}/ | jq .

`move_status` turns to `OK` and move_info has the result message

example)
```json
{
  "id": "turtlebot3",
  "type": "robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-07-18T10:00:14.00Z",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "executed triangle command",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T10:00:14.978Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T10:00:14.978Z"
      }
    }
  },
  "theta": {
    "type": "float32",
    "value": "0.4",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "x": {
    "type": "float32",
    "value": "0.1",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.2",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "z": {
    "type": "float32",
    "value": "0.3",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-07-18T17:41:15.1531903275+0900"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```